# Word info synthesis

In [1]:
from ipynb.fs.full.koselleck import *

In [2]:
def get_word_comparisons(word):
    with get_veclib('ttest') as vl_ttest_vec, get_veclib('ttest_lnm') as vl_ttest_lnm:
        wdf_ttest_vec=vl_ttest_vec.get(word,pd.DataFrame())
        wdf_ttest_lnm=vl_ttest_lnm.get(word,pd.DataFrame())
        odf=wdf_ttest_vec.append(wdf_ttest_lnm)
        if 'index' in set(odf.columns):odf=odf.drop('index',1)
        if len(odf):
            odf['avg_div']=odf.avg2/odf.avg1
            odf['avg_diff_perc']=odf.avg2_perc_vec - odf.avg1_perc_vec
            odf['avg_diff_perc_abs']=odf['avg_diff_perc'].apply(abs)
        return odf

In [3]:
get_word_comparisons('culture')

n1     n2      avg1      avg2  \
word    vector              period_cmp                                          
culture Abs-Conc.Median     Before-v-After    98.0  140.0  0.103371  1.684221   
                            Before-v-During   98.0  120.0  0.103371  0.274209   
                            During-v-After   120.0  140.0  0.274209  1.684221   
        Active-Passive.HGI  Before-v-After    98.0  140.0  0.081433 -0.233455   
                            Before-v-During   98.0  120.0  0.081433  0.648008   
                            During-v-After   120.0  140.0  0.648008 -0.233455   
        Ambig.NX            Before-v-After    98.0  140.0 -0.019773  0.181188   
                            Before-v-During   98.0  120.0 -0.019773 -0.300684   
                            During-v-After   120.0  140.0 -0.300684  0.181188   
        Collective-Indiv.RH Before-v-After    98.0  140.0  1.538172  1.621483   
                            Before-v-During   98.0  120.0  1.538172  1.645985   
                            During-v-After   120.0  140.0  1.645985  1.621483   
        Freq.M              Before-v-After    98.0  140.0 -0.234662  0.164169   
                            Before-v-During   98.0  120.0 -0.234662 -0.029103   
                            During-v-After   120.0  140.0 -0.029103  0.164169   
        Human-Object.VG     Before-v-After    98.0  140.0 -1.421982  0.197957   
                            Before-v-During   98.0  120.0 -1.421982 -1.441454   
                            During-v-After   120.0  140.0 -1.441454  0.197957   
        Interp-Desc.HGI     Before-v-After    98.0  140.0  0.652404  2.134646   
                            Before-v-During   98.0  120.0  0.652404  1.037795   
                            During-v-After   120.0  140.0  1.037795  2.134646   
        Judg-Perc.HGI       Before-v-After    98.0  140.0 -0.312116 -0.344856   
                            Before-v-During   98.0  120.0 -0.312116 -0.421600   
                            During-v-After   120.0  140.0 -0.421600 -0.344856   
        Pleasure-Pain.HGI   Before-v-After    98.0  140.0  0.884477  2.417986   
                            Before-v-During   98.0  120.0  0.884477  1.056648   
                            During-v-After   120.0  140.0  1.056648  2.417986   
        Polit-Acad.HGI      Before-v-After    98.0  140.0 -1.010255 -1.820035   
                            Before-v-During   98.0  120.0 -1.010255 -1.104238   
                            During-v-After   120.0  140.0 -1.104238 -1.820035   
        Pos-Neg.HGI         Before-v-After    98.0  140.0  1.045017  2.738036   
                            Before-v-During   98.0  120.0  1.045017  1.180521   
                            During-v-After   120.0  140.0  1.180521  2.738036   
        Qual-Quant.HGI      Before-v-After    98.0  140.0 -0.383044 -1.019468   
                            Before-v-During   98.0  120.0 -0.383044 -1.270962   
                            During-v-After   120.0  140.0 -1.270962 -1.019468   
        Sing-Plural.M       Before-v-After    98.0  140.0       NaN       NaN   
                            Before-v-During   98.0  120.0       NaN       NaN   
                            During-v-After   120.0  140.0       NaN       NaN   
        Strong-Weak.HGI     Before-v-After    98.0  140.0  0.806602  2.002279   
                            Before-v-During   98.0  120.0  0.806602  1.380181   
                            During-v-After   120.0  140.0  1.380181  2.002279   
        Time-Space.HGI      Before-v-After    98.0  140.0 -1.117554 -0.273421   
                            Before-v-During   98.0  120.0 -1.117554 -0.748454   
                            During-v-After   120.0  140.0 -0.748454 -0.273421   
        Virtue-Vice.HGI     Before-v-After    98.0  140.0  1.104495  2.998947   
                            Before-v-During   98.0  120.0  1.104495  1.245720   
                            During-v-After   120.0  140.0  1.245720  2.998947  

In [4]:
# get_word_comparisons('sympathy').loc['sympathy','LNM']

In [27]:
def info_word_comparisons(word,veckey='vector'):
    df=get_word_comparisons(word).reset_index().dropna()
    df.loc[df.vector=='LNM', 'avg_diff_perc'] *= -1
    
    most_changing_vecs = df.groupby(veckey).mean().reset_index().sort_values('avg_diff_perc_abs',ascending=False).drop_duplicates(veckey)[veckey]
#     print(most_changing_vecs)
    o=[]
    for vec in most_changing_vecs:
        if vec=='LNM': continue
        vecdf=df.query(f'vector=="{vec}"').set_index('period_cmp')
        try:

            if vec!='LNM':
                bef_dur=vecdf.loc['Before-v-During']
                dur_aft=vecdf.loc['During-v-After']
                bef_aft=vecdf.loc['Before-v-After']

            else:
                bef_dur=vecdf.loc['Before-v-Before']
                dur_aft=vecdf.loc['During-v-During']
                bef_aft=vecdf.loc['After-v-After']
        except KeyError:
            continue

        

    #         perc1,perc2,perc3 = bef_dur.avg1_perc_vec, bef_dur.avg2_perc_vec, dur_aft.avg2_perc_vec
        perc1,perc2,perc3 = bef_dur.avg1, bef_dur.avg2, dur_aft.avg2
        change=bef_aft.avg_diff
        is_signif=bef_aft.mw_p<.01
        mwperc=bef_aft.mw_perc
        mwp=bef_aft.mw_p
        
        
        vname=vec.split(".")[0]
        
        if '-' in vname:
            vname1,vname2=vname.split('-',1)
        else:
            vname1,vname2=vname,vname
        
        change2=abs(change) if vname2!=vname1 else change
        vname2_ch=vname2 if vname2!=vname1 else vname1
#         if vname=='LNM': change*=-1
        
        is_pos_change=change>0
        
        vname_change=vname1 if is_pos_change else vname2_ch
#         changestr=f'{change:+.0f}% {vname1}' if is_pos_change else f'{change2:+.0f}% {vname2_ch}'
        changestr=f'{abs(change):.2f}' if is_pos_change else f'{abs(change):.2f}'

        
#         x=f'* <u>{vname}</u>: {changestr} ({perc1:.0f}% -> {perc2:.0f}% -> {perc3:.0f}%)'
        vnameA = vname1 if perc1>0 else vname2
        vnameB = vname1 if perc2>0 else vname2
        vnameC = vname1 if perc3>0 else vname2
#         ltr1=f"{'+' if is_pos_change else '-'}"
        ltr1='+'
        vchange=f"""{ltr1 if vnameA!=vnameC else ''}{ltr1}{vname_change}"""
        if is_signif: 
            vchange=f'<b>{vchange}</b>'
            changestr=f'{changestr}'
        if vnameB!=vnameA: vnameB=f'<b>{vnameB}</b>'
        if vnameC!=vnameB: vnameC=f'<b>{vnameC}</b>'
            
#         perc1b=(100-perc1 if perc1>=50 else perc1)
#         perc2b=(100-perc2 if perc2>=50 else perc2)
#         perc3b=(100-perc3 if perc3>=50 else perc3)
#         perc1b=abs(perc1-50)*2
#         perc2b=abs(perc2-50)*2
#         perc3b=abs(perc3-50)*2
        
        
        
        
#         x=f'_{vname}_ | <u>{vchange}</u> ({changestr}) | {perc1:.0f} {perc1b:.0f}% {vnameA} | {perc2b:.0f}% {vnameB} | {perc3:.0f} {perc3b:.0f}% {vnameC}'
        x=f'_{vname}_ | <u>{vchange}</u> ({changestr}) | {perc1:+.2f} ({vnameA}) | {perc2:+.2f} ({vnameB}) | {perc3:+.2f} ({vnameC})'
        o+=[x]
    
    o=sorted(o,key=lambda x: float(x.split('|')[1].strip().split()[-1].replace(')','').replace('(','')),reverse=True)
    
    hdr="""
| Vector | Change | 1700-1770 | 1770-1830 | 1830-1900 |
| ------ | --------- | --------- | --------- | ------ |
    """
    ostr = hdr + '\n'.join(o)
    return ostr

printm(info_word_comparisons('culture'))    


| Vector | Change | 1700-1770 | 1770-1830 | 1830-1900 |
| ------ | --------- | --------- | --------- | ------ |
    _Virtue-Vice_ | <u><b>+Virtue</b></u> (1.89) | +1.10 (Virtue) | +1.25 (Virtue) | +3.00 (Virtue)
_Pos-Neg_ | <u><b>+Pos</b></u> (1.69) | +1.05 (Pos) | +1.18 (Pos) | +2.74 (Pos)
_Human-Object_ | <u><b>++Human</b></u> (1.62) | -1.42 (Object) | -1.44 (Object) | +0.20 (<b>Human</b>)
_Abs-Conc_ | <u><b>+Abs</b></u> (1.58) | +0.10 (Abs) | +0.27 (Abs) | +1.68 (Abs)
_Pleasure-Pain_ | <u><b>+Pleasure</b></u> (1.53) | +0.88 (Pleasure) | +1.06 (Pleasure) | +2.42 (Pleasure)
_Interp-Desc_ | <u><b>+Interp</b></u> (1.48) | +0.65 (Interp) | +1.04 (Interp) | +2.13 (Interp)
_Strong-Weak_ | <u><b>+Strong</b></u> (1.20) | +0.81 (Strong) | +1.38 (Strong) | +2.00 (Strong)
_Time-Space_ | <u><b>+Time</b></u> (0.84) | -1.12 (Space) | -0.75 (Space) | -0.27 (Space)
_Polit-Acad_ | <u><b>+Acad</b></u> (0.81) | -1.01 (Acad) | -1.10 (Acad) | -1.82 (Acad)
_Qual-Quant_ | <u><b>+Quant</b></u> (0.64) | -0.38 (Quant) | -1.27 (Quant) | -1.02 (Quant)
_Woman-Man_ | <u><b>+Man</b></u> (0.47) | -0.47 (Man) | -0.98 (Man) | -0.94 (Man)
_Freq_ | <u><b>+Freq</b></u> (0.40) | -0.23 (Freq) | -0.03 (Freq) | +0.16 (Freq)
_Active-Passive_ | <u><b>++Passive</b></u> (0.31) | +0.08 (Active) | +0.65 (Active) | -0.23 (<b>Passive</b>)
_Ambig_ | <u><b>+Ambig</b></u> (0.20) | -0.02 (Ambig) | -0.30 (Ambig) | +0.18 (Ambig)
_Collective-Indiv_ | <u>+Collective</u> (0.08) | +1.54 (Collective) | +1.65 (Collective) | +1.62 (Collective)
_Judg-Perc_ | <u>+Perc</u> (0.03) | -0.31 (Perc) | -0.42 (Perc) | -0.34 (Perc)

In [28]:
## Ranking magnitudes of change

In [29]:
# lnm_word('culture').mean()

In [30]:
# lnm_word('culture').lnm.describe()

In [31]:
# df=lnm_word('culture').reset_index()
# df.period1=df.period1.apply(lambda prd: periodize_sattelzeit_binary(int(prd[:4])))
# df.period2=df.period2.apply(lambda prd: periodize_sattelzeit_binary(int(prd[:4])))
# df=df.query('period1!=period2').dropna()
# s=df.lnm
# s.describe()

In [10]:
def do_describe_word_lnms(word):
    try:
        df=lnm_word(word).reset_index()
        df.period1=df.period1.apply(lambda prd: periodize_sattelzeit_binary(int(prd[:4])))
        df.period2=df.period2.apply(lambda prd: periodize_sattelzeit_binary(int(prd[:4])))
        df=df.query('period1!=period2').dropna()
        s=df.lnm
        s_out = (1-s).describe()
        for pn in range(5,100,5):
            s_out[f'{pn}%']=s.quantile(pn/100)
        return {word:s_out}
    except Exception as e:
#         print(f'!! Could not compute "{word}" because {e}')
        return pd.DataFrame()
from collections import ChainMap

def describe_word_lnms(words=None,num_proc=1):
    words=get_words_with_lnm() if not words else words
    o=pmap(do_describe_word_lnms, words, num_proc=num_proc)
    od=dict(ChainMap(*o))
    odf=pd.DataFrame(od).T
    for col in list(odf.columns): odf[col+'_rank']=odf[col].rank(ascending=False)
    odf=odf[sorted(list(odf.columns))]
    return odf.sort_values('50%_rank')

In [11]:
# do_describe_word_lnms('culture')

In [12]:
# do_describe_word_lnms('station')

In [13]:
# odf=describe_word_lnms(num_proc=4)

In [14]:
# FN_LNM_WORD_RANKING = os.path.join(PATH_DATA,'data.lnm_word_ranking.pkl')
# odf.to_pickle(FN_LNM_WORD_RANKING)

In [15]:
# odf.query('count>=100').sort_values('mean',ascending=False)[['mean','mean_rank']]

In [16]:
# vdf=lnm('virtue')
# vdf.sort_values('lnm')

In [23]:
# dfsimchanges=get_cdist_change_ranks()
# dfsimchanges

In [22]:
# printm(info_word_simchange('growth'))

In [19]:
df=get_cdist_change_ranks()


In [20]:
def info_word_markdown(w,percs=True):
    dirchange_md=format_tilts_sattelzeit_html(w,percs=percs)
    simchange_md=info_word_simchange(w)
    return HBox([
        markdwn(dirchange_md),
        markdwn(simchange_md)
    ])

In [26]:
# info_word_markdown('culture')